In [46]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD, RMSprop, Adam
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras import backend as K

import numpy as np

# TODO
- more aggresive data augmentation
- more aggressive dropout
- use of L1 and L2 regularization (also known as "weight decay")
- fine-tuning one more convolutional block (alongside greater regularization)


In [5]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
nb_train_samples = 5919
nb_validation_samples = 444
nb_epochs = 15
batch_size = 16
nb_classes = 3

In [3]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                  zoom_range=0.2,
                                  rotation_range=40, 
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 5919 images belonging to 3 classes.
Found 444 images belonging to 3 classes.


In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [8]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
itr = train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=5919,
    class_mode='categorical')

Found 5919 images belonging to 3 classes.


In [9]:
X, y = itr.next()

In [10]:
print(X.shape)
print(y.shape)

(5919, 224, 224, 3)
(5919, 3)


In [11]:
pre_processed_X = preprocess_input(X)

In [12]:
print(pre_processed_X.shape)
print(y.shape)

(5919, 224, 224, 3)
(5919, 3)


In [13]:
zero_centered_X = pre_processed_X

In [16]:
zero_centered_X -= np.mean(zero_centered_X, axis = 0) # zero-center the data (important)

In [17]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
print('Model loaded.')

Model loaded.


In [18]:
x = base_model.output
x = Flatten()(x)
#x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax', kernel_initializer='glorot_uniform')(x)

In [19]:
# this is the model we will train
# add your top layer block to your base model
model = Model(base_model.input, predictions)

In [20]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [37]:
def printAllLayers():
    # let's visualize layer names and layer indices to see how many layers
    # we should freeze:
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [21]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [22]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:19]:
    layer.trainable = False

In [35]:
def printTrainableLayers():
    # let's visualize layer names and layer indices to see how many layers
    # we should freeze:
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [23]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
   if layer.trainable == True: 
       print(i, layer.name)

19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [25]:
# checkpoint
filepath="vgg16-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [26]:
embedding_layer_list=['block5_pool', 'dense_10']

In [27]:
#histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [28]:
#opt = RMSprop(lr=0.1)
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#SGD(lr=1e-4, momentum=0.9)
#opt=SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Goal: < 0.96188 val_loss

In [32]:
history = model.fit(zero_centered_X, y, batch_size=16, epochs=50, callbacks=callbacks_list, validation_split=0.3, shuffle=True, initial_epoch=20)

Train on 4143 samples, validate on 1776 samples
Epoch 21/50
4143/4143 [==============================] - 71s - loss: 0.8449 - acc: 0.5267 - val_loss: 1.0203 - val_acc: 0.4972
Epoch 22/50
4143/4143 [==============================] - 71s - loss: 0.8424 - acc: 0.5257 - val_loss: 1.0156 - val_acc: 0.5203
Epoch 23/50
4143/4143 [==============================] - 108s - loss: 0.8551 - acc: 0.5141 - val_loss: 1.1116 - val_acc: 0.5073
Epoch 24/50
4143/4143 [==============================] - 100s - loss: 0.8393 - acc: 0.5356 - val_loss: 1.0390 - val_acc: 0.5130
Epoch 25/50
4143/4143 [==============================] - 70s - loss: 0.8437 - acc: 0.5284 - val_loss: 1.0556 - val_acc: 0.5214
Epoch 26/50
4143/4143 [==============================] - 82s - loss: 0.8346 - acc: 0.5281 - val_loss: 1.0272 - val_acc: 0.5130
Epoch 27/50
4143/4143 [==============================] - 70s - loss: 0.8191 - acc: 0.5358 - val_loss: 1.0479 - val_acc: 0.5225
Epoch 28/50
4143/4143 [==============================] - 75s 

In [33]:
# serialize model to JSON
model_json = model.to_json()
with open("model_vgg16_with_dropout.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_vgg162_with_dropout.h5")
print("Saved model to disk")

Saved model to disk


In [47]:
# load json and create model
json_file = open("model_vgg16_with_dropout.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("vgg16-weights-improvement-06-0.98.hdf5")

In [48]:
printAllLayers()

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [49]:
printTrainableLayers()

19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [50]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True    

In [51]:
printTrainableLayers()

15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dropout_1
22 dense_2


In [52]:
from keras.optimizers import SGD
opt=SGD(lr=1e-4, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy')

In [44]:
history = model.fit(zero_centered_X, y, batch_size=16, epochs=50, callbacks=callbacks_list, validation_split=0.3, shuffle=True, initial_epoch=0)

Train on 4143 samples, validate on 1776 samples
Epoch 1/50
4032/4143 [============================>.] - ETA: 5s - loss: 0.7587

KeyboardInterrupt: 